# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
api_key = geoapify_key
api_key

'43bc88720f674452a5025977075e5ce4'

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chabahar,51.5085,-0.1257,51.49,93,100,8.05,GB,1730927124
1,1,yukhnov,25.2919,60.6430,80.04,74,0,9.51,IR,1730927386
2,2,bilibino,54.7444,35.2297,43.23,92,100,11.72,RU,1730927387
3,3,tiksi,68.0546,166.4372,-10.35,100,100,0.96,RU,1730927388
4,4,abbeville,71.6872,128.8694,8.65,73,100,15.90,RU,1730927389


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display



city_map = city_data_df.hvplot.points(
    x='Lng', y='Lat', tiles='CartoLight',
    size='Humidity', color='Humidity', 
    cmap='cool', alpha=0.6, 
    frame_width=800, frame_height=500,
    title="City Locations with Humidity Levels"
)

city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [x,y]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[(city_data_df['Lat'] > 0) & (city_data_df['Max Temp'] > 70)]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,yukhnov,25.2919,60.6430,80.04,74,0,9.51,IR,1730927386
20,20,aqadyr,22.0669,-159.3780,81.39,86,20,19.57,US,1730927407
23,23,talas,8.4840,-13.2299,79.81,77,0,3.91,SL,1730927410
25,25,chibougamau,19.6406,-155.9956,82.36,79,20,10.36,US,1730927413
37,37,mirny,10.2753,170.8646,83.80,81,100,21.74,MH,1730927426
...,...,...,...,...,...,...,...,...,...,...
560,560,bahia de kino,24.1103,52.7306,79.86,66,0,9.62,AE,1730928043
561,561,poso,28.8333,-111.9333,73.63,47,0,8.79,MX,1730928044
563,563,qom,21.0500,71.7833,81.10,50,6,7.70,IN,1730928047
566,566,saynshand,18.4411,1.4078,73.89,18,0,10.25,ML,1730928050


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,chabahar,GB,51.5085,-0.1257,93,
1,yukhnov,IR,25.2919,60.6430,74,
2,bilibino,RU,54.7444,35.2297,92,
3,tiksi,RU,68.0546,166.4372,100,
4,abbeville,RU,71.6872,128.8694,73,
...,...,...,...,...,...,...
574,orkney,ES,42.4269,-2.4615,100,
575,auki,ZA,-26.9802,26.6727,15,
576,chaman,SB,-8.7676,160.7034,83,
577,plettenberg bay,PK,30.9236,66.4512,22,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel




# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    radius = 10000

    params = {
        'categories':'accommodation.hotel',
        'filter': f"circle:{longitude},{latitude},{radius}",
        "bias": f"proximity:{longitude},{latitude}",
        "limit": 1,
        "apiKey": api_key  

    
}

    # Add the current city's latitude and longitude to the params dictionary
    

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = response = requests.get(base_url,params=params)
    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
chabahar - nearest hotel: The Clermont
yukhnov - nearest hotel: هتل سپیده
bilibino - nearest hotel: Айсберг Угры
tiksi - nearest hotel: No hotel found
abbeville - nearest hotel: Арктика
blackmans bay - nearest hotel: Hôtel Mercure
ushuaia - nearest hotel: Villa Howden
iquique - nearest hotel: Apart Hotel Aires del Beagle
west island - nearest hotel: Hostal Vivar
ilulissat - nearest hotel: Cocos Village Bungalows
adamstown - nearest hotel: Best Western Plus Hotel Ilulissat
yellowknife - nearest hotel: No hotel found
qaqortoq - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Qaqortoq
bangjun - nearest hotel: Hotel Chathams
port-aux-francais - nearest hotel: No hotel found
invercargill - nearest hotel: Keravel
berezovyy - nearest hotel: The Grand
grytviken - nearest hotel: No hotel found
wailua homesteads - nearest hotel: No hotel found
aqadyr - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
olonkinbyen - nearest hotel: No hotel found
freetown - near

,City,Country,Lat,Lng,Humidity,Hotel Name
0,chabahar,GB,51.5085,-0.1257,93,The Clermont
1,yukhnov,IR,25.2919,60.6430,74,هتل سپیده
2,bilibino,RU,54.7444,35.2297,92,Айсберг Угры
3,tiksi,RU,68.0546,166.4372,100,No hotel found
4,abbeville,RU,71.6872,128.8694,73,Арктика
...,...,...,...,...,...,...
574,orkney,ES,42.4269,-2.4615,100,Hotel Iraipe
575,auki,ZA,-26.9802,26.6727,15,No hotel found
576,chaman,SB,-8.7676,160.7034,83,No hotel found
577,plettenberg bay,PK,30.9236,66.4512,22,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
hotel_df['Hover Info'] = (
    "City: " + hotel_df['City'] + "<br>" +
    "Country: " + hotel_df['Country'] + "<br>" +
    "Hotel: " + hotel_df['Hotel Name']
)

geo_plot = hotel_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True,  
    tiles='OSM',  
    hover_cols=['City', 'Country', 'Hotel Name'],  
    size=100,  
    color='blue',  
    title="City Locations with Nearby Hotels"
)

geo_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)

In [21]:
name_address = response.json()
name_address

{'statusCode': 400,
 'error': 'Bad Request',
 'message': '"filter[0]" does not match any of the allowed types'}

In [ ]:
%%capture --no-display

# Configure the map plot


# Display the map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)